In [9]:
# 业务逻辑不应关心底层是 Qwen 还是 Doubao —— 只需调用 .chat(messages)
# 定义一个基本类 BaseLLM，所有具体模型类都继承自它
from abc import ABC, abstractmethod
from typing import List, Dict

class BaseLLM(ABC):
    @abstractmethod
    def chat(self, messages: List[Dict[str, str]]) -> str:
        """向 LLM 发送对话历史，返回模型回复"""
        pass


In [10]:
# 实现 QwenLLM，使用 OpenAI 兼容接口调用 Qwen 模型
from openai import OpenAI
import os

class QwenLLM(BaseLLM):
    def __init__(self, api_key: str, model: str = "qwen-max"):
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
        )
        self.model = model
    
    def chat(self, messages):
        resp = self.client.chat.completions.create(
            model=self.model,
            messages=messages
        )
        return resp.choices[0].message.content

In [11]:
# 实现 DoubaoLLM，使用 OpenAI 兼容接口调用 豆包 模型
class DoubaoLLM(BaseLLM):
    def __init__(self, api_key: str, model: str = "doubao-seed-1-8-251228"):
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://ark.cn-beijing.volces.com/api/v3"
        )
        self.model = model
    
    def chat(self, messages):
        resp = self.client.chat.completions.create(
            model=self.model,
            messages=messages
        )
        return resp.choices[0].message.content

In [12]:
#  初始化两个模型（使用同一个 ARK_API_KEY？注意平台差异！）
qwen = QwenLLM(os.getenv("QWEN_API_KEY"))          # ← 百炼 Key
doubao = DoubaoLLM(os.getenv("ARK_API_KEY"))       # ← 火山方舟 Key

messages = [{"role": "user", "content": "用一句话解释机器学习"}]

print("🤖 Qwen:", qwen.chat(messages))
print("🤖 Doubao:", doubao.chat(messages))

🤖 Qwen: 机器学习是让计算机通过数据和经验自动改进和优化算法性能的一种人工智能技术。
🤖 Doubao: 机器学习是人工智能的一个分支，它无需人类为计算机编写处理任务的明确规则，而是让计算机通过分析海量数据自主学习规律与模式，从而具备对未知数据进行预测、分类或决策的能力。


In [13]:
# 封装模型工厂函数
def create_llm(provider: str, **kwargs) -> BaseLLM:
    if provider == "qwen":
        return QwenLLM(kwargs["api_key"], kwargs.get("model", "qwen-max"))
    elif provider == "doubao":
        return DoubaoLLM(kwargs["api_key"], kwargs.get("model", "doubao-seed-1-8-251228"))
    else:
        raise ValueError(f"Unsupported provider: {provider}")

# 使用
llm = create_llm("qwen", api_key=os.getenv("QWEN_API_KEY"))
print(llm.chat([{"role": "user", "content": "你好"}]))

你好！有什么我可以帮助你的吗？
